# ProcurementPro API Testing
This notebook contains test cases for the ProcurementPro API endpoints.

## Setup

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt

# Load environment variables
load_dotenv()

# API Configuration
BASE_URL = os.getenv('API_BASE_URL', 'http://localhost:3001/api/v1')
HEADERS = {
    'Content-Type': 'application/json'
}

# Test user credentials
TEST_USER = {
    'email': 'test@example.com',
 'password': 'testpassword123'
}

# Helper function to make authenticated requests
def get_auth_headers(token):
    return {**HEADERS, 'Authorization': f'Bearer {token}'}

## Authentication Tests

In [ ]:
def test_user_login() -> str:
    """Test user login and return access token"""
    try:
        response = requests.post(
            f'{BASE_URL}/auth/login',
            headers=HEADERS,
            json=TEST_USER
        )
        response.raise_for_status()
        token = response.json().get('token')
        print("✅ Login successful")
        return token
    except requests.exceptions.RequestException as e:
        print(f"❌ Login failed: {e}")
        return None

## API Endpoint Tests

In [ ]:
def test_get_loans(token: str):
    """Test getting list of loans"""
    try:
        response = requests.get(
            f'{BASE_URL}/loans',
            headers=get_auth_headers(token)
        )
        response.raise_for_status()
        loans = response.json()
        print(f"✅ Retrieved {len(loans)} loans")
        return loans
    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to get loans: {e}")
        return None

## Run All Tests

In [ ]:
def run_all_tests():
    print("🚀 Starting API Tests...")
    
    # Test authentication
    token = test_user_login()
    if not token:
        print("❌ Authentication tests failed")
        return
    
    # Test API endpoints
    loans = test_get_loans(token)
    if loans is not None:
        # Convert to DataFrame for better visualization
        df = pd.DataFrame(loans)
        if not df.empty:
            display(df.head())
    
    print("✅ All tests completed")

# Run all tests
run_all_tests()

## Performance Testing

In [ ]:
def test_endpoint_performance(endpoint: str, method='GET', token=None, payload=None):
    """Test endpoint performance"""
    import time
    
    headers = get_auth_headers(token) if token else HEADERS
    url = f'{BASE_URL}/{endpoint}'
    
    try:
        start_time = time.time()
        if method.upper() == 'GET':
            response = requests.get(url, headers=headers)
        elif method.upper() == 'POST':
            response = requests.post(url, headers=headers, json=payload)
        elif method.upper() == 'PUT':
            response = requests.put(url, headers=headers, json=payload)
        elif method.upper() == 'DELETE':
            response = requests.delete(url, headers=headers)
            
        response_time = (time.time() - start_time) * 1000  # Convert to milliseconds
        response.raise_for_status()
        
        return {
            'endpoint': endpoint,
 'method': method,
 'status_code': response.status_code,
 'response_time_ms': response_time,
 'success': True
        }
    except requests.exceptions.RequestException as e:
        return {
            'endpoint': endpoint,
 'method': method,
 'status_code': getattr(e.response, 'status_code', None) if hasattr(e, 'response') else None,
 'error': str(e),
 'success': False
        }

In [ ]:
def run_performance_tests(token):
    endpoints = [
        ('auth/me', 'GET', token),
        ('loans', 'GET', token),
        # Add more endpoints to test
    ]
    
    results = []
    for endpoint, method, t in endpoints:
        result = test_endpoint_performance(endpoint, method, t)
        results.append(result)
    
    # Display results
    df = pd.DataFrame(results)
    if not df.empty:
        display(df)
        
        # Plot response times
        if not df[df['success']].empty:
            plt.figure(figsize=(10, 5))
            df[df['success']].plot.bar(
                x='endpoint',
                y='response_time_ms',
                title='API Response Times (ms)',
                legend=False
            )
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()

In [ ]:
# Run performance tests with authenticated user
token = test_user_login()
if token:
    run_performance_tests(token)

## Load Testing (Sample)

In [ ]:
def run_load_test(endpoint: str, num_requests: int = 10, token=None):
    """Run a simple load test on an endpoint"""
    import time
    import statistics
    
    headers = get_auth_headers(token) if token else HEADERS
    url = f'{BASE_URL}/{endpoint}'
    
    response_times = []
    success_count = 0
    
    print(f"🚀 Starting load test for {endpoint} with {num_requests} requests...")
    
    for i in range(num_requests):
        try:
            start_time = time.time()
            response = requests.get(url, headers=headers)
            response_time = (time.time() - start_time) * 1000  # Convert to ms
            response_times.append(response_time)
            
            if response.status_code == 200:
                success_count += 1
                
            if i % 10 == 0:
                print(f"Completed {i+1}/{num_requests} requests")
                
        except Exception as e:
            print(f"Error on request {i+1}: {str(e)}")
    
    # Calculate statistics
    if response_times:
        stats = {
            'endpoint': endpoint,
 'total_requests': num_requests,
 'successful_requests': success_count,
 'success_rate': (success_count / num_requests) * 100,
 'avg_response_time_ms': statistics.mean(response_times),
 'min_response_time_ms': min(response_times),
 'max_response_time_ms': max(response_times),
 'p95_response_time_ms': sorted(response_times)[int(len(response_times) * 0.95)]
        }
        
        # Display results
        df = pd.DataFrame([stats])
        display(df)
        
        # Plot response time distribution
        plt.figure(figsize=(10, 5))
        plt.hist(response_times, bins=20, alpha=0.7, color='blue')
        plt.axvline(statistics.mean(response_times), color='red', linestyle='dashed', linewidth=1)
        plt.title(f'Response Time Distribution for {endpoint}')
        plt.xlabel('Response Time (ms)')
        plt.ylabel('Frequency')
        plt.show()
        
        return stats
    else:
        print("❌ No successful requests to analyze")
        return None

In [ ]:
# Example: Run load test on the loans endpoint
# Note: Adjust the number of requests based on your testing needs
# run_load_test('loans', num_requests=50, token=token)